## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_high = 600
cutoff_low = 20
application_types_to_replace = []
for item in application_df['APPLICATION_TYPE'].value_counts().items():
    if item[1] < cutoff_high and item[1] > cutoff_low:
        application_types_to_replace.append(item[0])
    elif item[1] < cutoff_low:
        application_types_to_replace.append(item[0])
        

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_high = 500
cutoff_low = 2
classifications_to_replace = []
for item in application_df['CLASSIFICATION'].value_counts().items():
    if item[1] < cutoff_high and item[1] > cutoff_low:
        classifications_to_replace.append(item[0])
    elif item[1] < cutoff_low:
        classifications_to_replace.append(item[0])
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1474
C7000      777
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: count, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(data=application_df, columns=list(set(['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION',
                                                             'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS',
                                                             'INCOME_AMT'])-set(['EIN', 'NAME'])))

In [15]:
# Split our preprocessed data into our features and target arrays
y = converted_data['IS_SUCCESSFUL']
X = converted_data.drop(columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.25)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = Sequential()

# First hidden layer
nn.add(Dense(units=90, activation = 'relu', input_dim=X_test.shape[1]))

# Second hidden layer
nn.add(Dense(units=20, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                4410      
                                                                 
 dense_1 (Dense)             (None, 20)                1820      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 6251 (24.42 KB)
Trainable params: 6251 (24.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [21]:
#Checkpoint
checkpoint = ModelCheckpoint("best_model_checkpoint.h5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=40, callbacks=[checkpoint])

Epoch 1/40
  1/804 [..............................] - ETA: 7:05 - loss: 0.7025 - accuracy: 0.3125
Epoch 1: accuracy improved from -inf to 0.45000, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.45000 to 0.50937, saving model to best_model_checkpoint.h5
 10/804 [..............................] - ETA: 4s - loss: 0.6857 - accuracy: 0.5094  
Epoch 1: accuracy improved from 0.50937 to 0.56667, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.56667 to 0.59844, saving model to best_model_checkpoint.h5
 20/804 [..............................] - ETA: 4s - loss: 0.6726 - accuracy: 0.5984
Epoch 1: accuracy improved from 0.59844 to 0.62750, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.62750 to 0.64167, saving model to best_model_checkpoint.h5
 30/804 [>.............................] - ETA: 4s - loss: 0.6598 - accuracy: 0.6417
Epoch 1: accuracy improved from 0.64167 to 0.65625, saving model to best_model_checkpoi

C:\Users\dawso\anaconda3\envs\dev\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 40/804 [>.............................] - ETA: 4s - loss: 0.6449 - accuracy: 0.6625
Epoch 1: accuracy improved from 0.66250 to 0.66875, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.66875 to 0.67500, saving model to best_model_checkpoint.h5
 50/804 [>.............................] - ETA: 4s - loss: 0.6391 - accuracy: 0.6750
Epoch 1: accuracy did not improve from 0.67500

Epoch 1: accuracy improved from 0.67500 to 0.67604, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.67604 to 0.68413, saving model to best_model_checkpoint.h5
 65/804 [=>............................] - ETA: 3s - loss: 0.6294 - accuracy: 0.6841
Epoch 1: accuracy improved from 0.68413 to 0.68437, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.68437 to 0.68917, saving model to best_model_checkpoint.h5
 76/804 [=>............................] - ETA: 3s - loss: 0.6224 - accuracy: 0.6891
Epoch 1: accuracy improved from 0.68917 to 0.69023,

531/804 [==================>...........] - ETA: 0s - loss: 0.5741 - accuracy: 0.7190
Epoch 1: accuracy did not improve from 0.72003

Epoch 1: accuracy did not improve from 0.72003

Epoch 1: accuracy improved from 0.72003 to 0.72053, saving model to best_model_checkpoint.h5

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053
555/804 [===================>..........] - ETA: 0s - loss: 0.5736 - accuracy: 0.7199
Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053
592/804 [=====================>........] - ETA: 0s - loss: 0.5735 - accuracy: 0.7194
Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not improve from 0.72053

Epoch 1: accuracy did not imp


Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488
377/804 [=============>................] - ETA: 0s - loss: 0.5502 - accuracy: 0.7312
Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488
414/804 [==============>...............] - ETA: 0s - loss: 0.5515 - accuracy: 0.7311
Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accuracy did not improve from 0.76488

Epoch 2: accura

...


Epoch 37: accuracy did not improve from 0.85156
 89/804 [==>...........................] - ETA: 0s - loss: 0.5388 - accuracy: 0.7395
Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156
135/804 [====>.........................] - ETA: 0s - loss: 0.5319 - accuracy: 0.7456
Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156


Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156

Epoch 37: accuracy did not improve from 0.85156
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7380
Epoch 38/40
  1/804 [..............................] - ETA: 1s - loss: 0.4805 - accuracy: 0.7812
Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156
 43/804 [>.............................] - ETA: 0s - loss: 0.5350 - accuracy: 0.7398
Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85

676/804 [========================>.....] - ETA: 0s - loss: 0.5394 - accuracy: 0.7369
Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156
725/804 [==========================>...] - ETA: 0s - loss: 0.5394 - accuracy: 0.7370
Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156

Epoch 38: accuracy did not improve from 0.85156


Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156
585/804 [====================>.........] - ETA: 0s - loss: 0.5386 - accuracy: 0.7377
Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156
630/804 [======================>.......] - ETA: 0s - loss: 0.5412 - accuracy: 0.7360
Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156

Epoch 39: accuracy did not improve from 0.85156


Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156
464/804 [================>.............] - ETA: 0s - loss: 0.5341 - accuracy: 0.7419
Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156
513/804 [==================>...........] - ETA: 0s - loss: 0.5351 - accuracy: 0.7411
Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

Epoch 40: accuracy did not improve from 0.85156

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5523 - accuracy: 0.7303 - 275ms/epoch - 1ms/step
Loss: 0.5522632002830505, Accuracy: 0.7302623987197876


In [24]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')